<a href="https://colab.research.google.com/github/VisionLogic-AI/Unsupervised_learning/blob/master/Unsupervised_Learning_(Group_Segmentation)_ipnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Group Segmentation 
From an applied perspective, the ability to segment members into a group based on similarity and without any guidance frokm labels is very poweful.<br>
For example) Finding different consumer groups for online retailers, customizing a marketing strategy for each distinct group etc.

*The options are endless for finding diffent customer segment groups once your data has has dimensionality reduction applied to it and then cluster with great accuracy....all without using any labels!

#Lending Club Data
In this notebook we will use loan data frokm Lending Club, a US peer to peer lending company

In [ ]:
#download github repo 
!git clone https://github.com/aapatel09/handson-unsupervised-learning.git

Cloning into 'handson-unsupervised-learning'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 459 (delta 14), reused 21 (delta 5), pack-reused 410
Receiving objects: 100% (459/459), 93.79 MiB | 28.99 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [ ]:
#implement the correct version of tensorflow
%tensorflow_version 1.x
!pip install tensorflow 
import tensorflow as tf
print(tf.__version__)

1.15.2


In [ ]:
#Install our other dependencies
!pip install keras
!pip install xgboost
!pip install lightgbm
!pip install fastcluster
!pip install hdbscan
!pip install tslearn

     |████████████████████████████████| 163kB 3.4MB/s 
     |████████████████████████████████| 4.7MB 3.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.26-cp36-cp36m-linux_x86_64.whl size=2307189 sha256=3a4c018e92cfbcd78fb3e4e284b3e3515a9e2d225a3c5a1a20f9ae956320f40e
  Stored in directory: /root/.cache/pip/wheels/82/38/41/372f034d8abd271ef7787a681e0a47fc05d472683a7eb088ed
Successfully built hdbscan
     |████████████████████████████████| 778kB 3.5MB/s 


#Data Preparation
Lets prepare our environment to work with the Lending Club data

In [5]:
#load libs
import numpy as np
import pandas as pd
import os, time, re
import pickle, gzip

#Data Viz
import matplotlib.pyplot as plt
import seaborn as sns
color= sns.color_palette()
import matplotlib as mpl

%matplotlib inline

#Data Prep and Model Evaluation
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score

#Alogorithms
from sklearn.decomposition import PCA  
from sklearn.cluster import KMeans
import fastcluster
from scipy.cluster.hierarchy import dendrogram, cophenet, fcluster
from scipy.spatial.distance import pdist

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#Explore Data
Now lets load the loan data and designate which of the columns to keep.<br>

The original loan data has 144 columns, but most of these columns are empty and are of little value to us.

In [47]:
#load data
data = pd.read_csv('/content/handson-unsupervised-learning/datasets/lending_club_data/LoanStats3a.csv')

#select columns to keep while deleting the remaining non value columns (the opposite of manually removing columns one by one)
data.drop(data.columns.difference(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
                'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership',
                 'annual_inc', 'verification_status', 'pymnt_plan', 'purpose', 'addr_state',
                 'dti', 'deling_2yrs', 'earliest_cr_line', 'mths_since_last_delinq', 'mths_since_last_record',
                 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status',
                 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
                 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
                 'last_pymnt_d', 'last_pymnt_amnt']), 1, inplace= True)
data.shape
data.head(5)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,addr_state,dti,earliest_cr_line,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt
0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,10+ years,RENT,24000.0,Verified,n,credit_card,AZ,27.65,Jan-85,NaN,NaN,3.0,0.0,13648.0,83.70%,9.0,f,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,Jan-15,171.62
1,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,< 1 year,RENT,30000.0,Source Verified,n,car,GA,1.00,Apr-99,NaN,NaN,3.0,0.0,1687.0,9.40%,4.0,f,0.0,0.0,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,Apr-13,119.66
2,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,10+ years,RENT,12252.0,Not Verified,n,small_business,IL,8.72,Nov-01,NaN,NaN,2.0,0.0,2956.0,98.50%,10.0,f,0.0,0.0,3005.666844,3005.67,2400.00,605.67,0.00,0.0,0.00,Jun-14,649.91
3,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,10+ years,RENT,49200.0,Source Verified,n,other,CA,20.00,Feb-96,35.0,NaN,10.0,0.0,5598.0,21%,37.0,f,0.0,0.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.0,0.00,Jan-15,357.48
4,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,1 year,RENT,80000.0,Source Verified,n,other,OR,17.94,Jan-96,38.0,NaN,15.0,0.0,27783.0,53.90%,38.0,f,0.0,0.0,4066.908161,4066.91,3000.00,1066.91,0.00,0.0,0.00,Jan-17,67.30


#Transform String Format to "Numerical" Format
A few of the features need to be altered from a string format to a numerical format.

In [48]:
#transform features from string to numeric 
for i in ['term', 'int_rate', 'emp_length', 'revol_util']:
  data.loc[:, i]= data.loc[:, i].apply(lambda x: re.sub('[^0-9]','', str(x)))
  data.loc[:, i]= pd.to_numeric(data.loc[:, i])

For our clustering application, we will consider only the numercial features and ignore all of the categorical features because non-numerical features cannot be handled by our clustering algorithm in their current form.

#Impute Missing Values
Lets find these numerical features and count the number of NAN's per feature.<br>
We will then impute these NAN 's with either a mean of the feature or, in some cases, just the number zero, depending on what feature represents from a business perspectives:

In [50]:
#Determine which features are numerical
numerical_feats = [x for x in data.columns if data [x].dtype != 'object']

#Display NaN's by feature
nan_counter= np.isnan(data.loc[:, numerical_feats]).sum()
print(nan_counter)

loan_amnt                      7
funded_amnt                    7
funded_amnt_inv                7
term                           7
int_rate                       7
installment                    7
emp_length                  1119
annual_inc                    11
dti                            7
mths_since_last_delinq     26933
mths_since_last_record     38891
open_acc                      36
pub_rec                       36
revol_bal                      7
revol_util                    97
total_acc                     36
out_prncp                      7
out_prncp_inv                  7
total_pymnt                    7
total_pymnt_inv                7
total_rec_prncp                7
total_rec_int                  7
total_rec_late_fee             7
recoveries                     7
collection_recovery_fee        7
last_pymnt_amnt                7
dtype: int64


Most of our features have NAN values and some have many.<br>

Lets impute these so we do not have to deal with any NaNs during the clutering process.

In [55]:
from sklearn.impute import SimpleImputer

#Impute NaNs with mean
fill_with_mean= ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term',
                 'int_rate', 'installment', 'emp_length', 'annual_inc',
                 'dti', 'open_acc', 'revol_bal', 'revol_util', 'total_acc', 
                 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 
                 'total_rec_prncp', 'total_rec_int', 'last_pymnt_amnt']

#impute Nans with zero
fill_with_zero= ['delinq_2yrs', 'mths_since_last_delinq', 'mths_since_last_record', 
                 'pub_rec', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee']

#Perform imputation ( I may have to find a different solution instead of using .loc )
im= SimpleImputer(strategy= 'mean')
data.loc[:, fill_with_mean]= im.fit_transform(data[fill_with_mean])
data.loc[:, fill_with_zero]= data.loc[:, fill_with_zero].fillna(value= 0, axis= 1)

KeyError: ignored

Lets recalculate the NaNs to mke sure no Nans remain.

In [ ]:
numerical_feats= [x for x in data.columns if data [x].dtype != 'object']
nan_counter= np.isnan(data.loc[:, numerical_feats]).sum()

#Engineering Features
Lets also engineer a few more features to add to the existing feature set.<br>
These new features are mostly ratios between loan amount, revolving balance, payments and the borrowers annual income:

In [ ]:
#Feature Engineering
data['installmentOverLoanAmnt']= data.installment/data.loan_amnt
data['loanAmntOverIncome']= data.loan_amnt/data.annual_inc
data['revol_balOverIncome']= data.revol_bal/data.annual_inc
data['totalPymntOverIncome']= data.total_pymnt/ data.annual_inc
data['totalPymntInvOverIncome']= data.total_pymnt_inv/data.annual_inc
data['totalRecPrncpOverIncome']= data.total_rec_prrncp/data.annual_inc
data['totalRecIncOverIncome']= data.total_rec_int/data.annual_inc/

new_feats= ['installmentOverLoanAmnt', 'loanAmntOverIncome',
            'revol_balOverIncome', 'totalPymntOverIncome',
            'totalPymntInvOverIncome', 'totalRecPrncpOverIncome', 
            'totalRecIncOverIncome']

In [ ]:
#Select Features for Training
numerical_plus_new_feats= numerical_feats + new_feats
x_train= data.loc[:, numerical_plus_new_feats]

#Scale data
sx= pp.StandardScaler()
x_train.loc[:, :]= sx.fit_transform(x_train)

In [ ]:
x_train.columns

In [ ]:
#Designate Labels for Evaluation
labels= data.grade
labels.unique()

In [ ]:
#Fill missing labels
labels= labels.fillna(value= 'Z')

#Convert labels to numerical values
lbl= pp.LabelEncoder()
lbl.fit(list(labels.values))
labels= pd.Series(data= lbl.transform(labels.values), name= 'grade')

#Store as y_train
y_train= labels

In [ ]:
labels_original_vs_news= pd.concat([labels, data.grade], axis= 1)
labels_original_vs_news

In [ ]:
#Compare Loan grades with interest rates
interest_and_grade= pd.DataFrame(data= [data.int_rate, labels])
interest_and_grade= interest_and_grade.T

interest_and_grade.groupby('grade').mean()

In [ ]:
#Goodness of the clusters

In [ ]:
def analyzeCluster(clusterDF, labelsDF):
  countbycluster= pd.DataFrame(data= clusterDF['cluster'].value_counts())
  countbycluster.reset_index(inplace= True, drop= False)
  countbycluster.columns= ['cluster', 'cluster_count']

  preds= pd.concat([labelsDF, clusterDF], axis= 1)
  preds.columns= ['truelabel', 'cluster']

  countbylabel= pd.DataFrame(data= preds.groupby('truelabel').count())

  countmostfreq= pd.DataFrame(data= preds.groupby('cluster').agg(lambda x:x.value_counts().iloc[0]))
  countmostfreq.reset_index(inplace= True, drop= False)
  countmostfreq.columns= ['cluster', 'countmostfrequent']

  accuracyDF= countmostfreq.merge(countbycluster, left_on= 'cluster', right_on= 'cluster')

  overallaccuracy= accuracyDF.countmostfrequent.sum()/ accuracyDF.clustercount.sum()

  accuracybylabel= accuracyDF.countmostfrequent/ accuracyDF.clustercount

  return countbycluster, countbylabel, countmostfreq, accuracyDF, overallaccuracy, accuracybylabel

#K-Means Clustering Implementation

In [ ]:
#k_Means Application
from sklearn.cluster import KMeans

n_clusters = 10
n_init= 10
max_iter= 300
tol= 0.0001
random_state= 2018
n_jobs= 2

kmeans= KMeans(n_clusters= n_clusters, n_init= n_init, max_iter= max_iter,
               tol= tol, random_state= random_state, n_jobs= n_jobs)

kmeans_inertia= pd.DataFrame(data= [], index= range(10,31), 
                             columns= ['inertia'])

overallaccuracy_kmeansDF= pd.DataFrame(data= [], index= range(10, 31), columns= ['overall_accuracy'])

for n_clusters in range(10,31):
  kmeans= KMeans(n_clusters= n_clusters, n_init= n_init,  max_iter= max_iter, tol= tol,
                 random_state= random_state, n_jobs = n_jobs)
  kmeans.fit(x_train)
  kmeans_inertia.loc[n_clusters]= kmeans.inertia_
  x_train_kmeansclustered= kmeans.predict(x_train)
  x_train_kmeansclustered= pd.DataFrame(data= x_train_kmeansclustered, index= x_train.index,
                                        columns= ['cluster'])
  
  countbycluster_kmeans, countbylabel_kmeans, \
  countmostfreq_kmeans, accuracyDF_kmeans, \
  overallaccuracy_kmeans, accuracybylabel_kmeans= analyzecluster(x_train_kmeans_clustered, y_train)

  overallaccuracy_kmeansDF.loc[n_clusters]= overallaccuracy_kmeans

In [ ]:
overallaccuracy_kmeansDF.plot()

In [ ]:
accuracybylabel_kmeans

#Hierarchical Clustering Implementation

In [ ]:
import fastcluster
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

Z= fastcluster.linkage_vector(x_train, method= 'ward', metric= 'euclidean')
z_dataframe= pd.DataFrame(data= Z, columns= ['cluster_one', 'cluster_two'.
                                             'distance', 'New Cluster Size'])

In [ ]:
z_dataframe[:20]

In [ ]:
z_dataframe[42521:]

In [ ]:
from scipy.cluster.hierarchy import fcluster

distance_threshold= 100
clusters= fcluster(Z, distance_threshold, criterion= 'distance')
x_train_hierclustered= pd.DataFrame(data = clusters, index= x_train.index, columns= ['cluster'])

print('Number of distinct clusters: ', len(x_train_hierclustered['cluster'].unique()))

In [ ]:
countbycluster_hierclust, countbylabel_hierclust, countmostfreq_hierclust,
accuracy_hierclust, overallaccuracy_hierclust, accuracybylabel_hierclust= analyze_cluster(x_train_hierclustered, y_train)

print('Overall Accuracy from hierarchical clustering: ', overallaccuracy_hierclust)

#HDBSCAN Implementation

In [ ]:
#hdbscan
import hdbscan

min_cluster_size= 20
min_samples= 20
alpha= 1.0
cluster_selection_method= 'leaf'

hdb= hdbscan.HDBSCAN(min_cluster_size= ,min_cluster_size, min_samples= min_samples,
                     alpha = alpha, cluster_selection_method= cluster_selection_method)

x_train_hdbscanclustered= hdb.fit_predict(x_train)
x_train_hdbcanclustered= pd.DataFrame(data= x_train_hdbscanclustered, index= x_train.index,
                                      columns= ['cluster'])

countbycluster_hdbscan, countybylabel_hdbscan, countmostfreq_hdbscan, 
accuracyDF_hdbscan, overallaccuracy_hdbscan, accuracybylabel_hdbscan= analyzecluster(x_train_hdbscanclustered, y_train)

print('Overall accuracy for HDBSCAN: ', overallaccuracy_hdbscan)
print('Cluster Results for HDBSCAN', countbycluster_hdbscan)

In [ ]:
accuracybylabel_hdbscan